# Arbitrage Detector Agent

The ``ArbitrageDetectorAgent` monitors price discrepancies between two liquidity pools (e.g., Uniswap V2 and another exchange) for the same token pair and sends notifications when the difference exceeds a specified percentage threshold, highlighting potential arbitrage opportunities after accounting for fee

### 📘 Notable Classes

---

* **Class**: 📘 ``defipy.agents.config.ArbitrageDetectorConfig`` 
    * **Purpose**: Configuration for volume threshold agent.
        * **Parameters**:
            * `volume_threshold`: Volume threshold to check (`float`).
            * `pool_address`: UniV2 pool address (`str`).
            * `provider_url`: Provider URL (eg, infura.io)(`str`).
            * `platform`: Platform where pool resides (eg, uniswap) (`str`).
            * `abi_name`: ABI handle pointing to pool contract in JSON (eg, UniswapV2Pair) (`str`).

---
         
* **Class**: 📘 ``defipy.agents.ArbitrageDetectorAgent`` 
    * **Purpose**: Reactive DeFi agent for determining price threshold.
        * **Parameters**:
            * `config`: Agent configuration parameters (`PriceThresholdConfig`).
    * **Methods**:
        * ``apply()``
            * **Purpose**: Apply price threshold agent
        * ``run_batch(lp: UniswapExchange, tkn: ERC20, user_nm: str, events: dict)``
            * **Purpose**: Run AI price agent on batch data 
            * **Parameters**:
                * `lp`: Swap token (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `events`: Dictionary of sync events (`dict`).
        * ``apply(lp: UniswapExchange, tkn: ERC20, user_nm: str, block_num: int)``
            * **Purpose**: Apply TVL check
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `block_num`: Block number (`int`).             
        * ``check_condition(lp: UniswapExchange, tkn: ERC20, threshold: float, block_num: int)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `threshold`: override config price threshold (optional) (`float`).
                * `block_num`: Block number (`int`).
        * ``get_pool_volume(lp: UniswapExchange, tkn: ERC20, block_num: int)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `block_num`: Block number (`int`).
        * ``take_mock_position(lp: UniswapExchange, tkn: ERC20, user_nm: str, amt: float)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `amt`: Mock token amount (`float`).
        * ``withdraw_mock_position(lp: UniswapExchange, tkn: ERC20, user_nm: str, lp_amt: float)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `lp_amt`: Mock LP amount (`float`).
        * ``update_mock_pool(lp: UniswapExchange, cur_block: int)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `cur_block`: Current block number (`int`).
        * ``prime_mock_pool(start_block: int, user_nm: str)``
            * **Purpose**: Initialize off-chain pool using information in start_block
            * **Parameters**:
                * `start_block`: Start block (`int`).
                * `user_nm`: Account name of mock off-chain pool (`str`). 
        * ``get_w3()``
            * **Purpose**: Getter function for web3.py connector object
        * ``get_abi()``
            * **Purpose**: Getter function for ABI json data
        * ``get_contract_instance()``
            * **Purpose**: Getter function for pool contract instance
        * ``get_lp_data()``
            * **Purpose**: Getter function for pool data associated with `pool_address` from config


In [1]:
from defipy import *
from web3scout import *

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/eth_tester/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
volume_threshold = 1000
pair_address1 = "0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc"
#pair_address2 = "0x397ff1542f962076d0bfe58ea045ffa2d347aca0"
provider_url = "https://mainnet.infura.io/v3/9624e3e5c40f4ac3958b79fa5aa2562d"
platform = Platform.AGNOSTIC
abi_name = JSONContract.UniswapV2Pair
user_position = 10

config = ArbitrageDetectorConfig(
    volume_threshold = volume_threshold,
    pool_address = pair_address,
    provider_url = provider_url,
    platform = platform,
    abi_name = abi_name,
    user_position = user_position
)

agent = ArbitrageDetectorAgent(config)
agent.init()

print(f"Monitoring TVL changes @ pool address {pair_address}")

NameError: name 'ArbitrageDetectorConfig' is not defined

In [ ]:
#pair_address = "0x397ff1542f962076d0bfe58ea045ffa2d347aca0"

In [ ]:
abi = ABILoad(platform, abi_name)
connect = ConnectW3(provider_url)
connect.apply()

last_block = connect.get_w3().eth.block_number
start_block = last_block - 100

# Grab batch sync events from pool
rEvents = RetrieveEvents(connect, abi)
events = rEvents.apply(EventType.SWAP, address = pair_address, start_block=start_block, end_block=last_block)
df_events = rEvents.to_dataframe(events)
df_events.head(2)

### Agent Run #1

In [ ]:
tkn0 = agent.get_lp_data().tkn0
tkn1 = agent.get_lp_data().tkn1
lp = agent.prime_mock_pool(start_block, 'user')
lp.summary()

In [ ]:
agent.run_batch(lp, tkn0, 'user', events)